### Objective - Event Registration Agent
- Allows users to navigate topics available in a seminar and to register

In [ ]:
# Basic Imports
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [10]:
# Define Agent State
from typing import Annotated
from typing_extensions import TypedDict
import operator
from langgraph.graph.message import add_messages
from langchain_core.messages import  AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from typing import Literal

class RegisterState(TypedDict):
    """ State representing User interaction with Event Registration Agent that persists all conversations between human, agent and function calls. """
    messages: Annotated[list[AnyMessage], operator.add]
    registered_events : list[str]
    finished_conversation : bool


system_prompt = """ 
You are a helpful ai assistant that helps users register for events going to be held on June 1, 2025 at Hunt Valley, MD, USA - 21030. The name of the event is - Generative AI for Software Developers.
You will be provided with a list of events and their details.
You will ask the user for their name and email address.
You will then ask the user which events they would like to register for.
You shall call event_menu function to get the list of events and show them to the user.
Once the user selects an event, you will call the function add_registration to register them for the event.
You will then call the function get_registration to fetch the registration details.
You will then ask the user to confirm their registration.
You will call the function confirm_registration to confirm the registration.
You will then ask the user if they would like to register for any other events.
If the user wants to cancel the registration for an event, you will call the function delete_registration.
If the user wants to modify the registration for an event that the user had registered before, then just delete the previous registration by calling function delete_registration and create a new one by calling function add_registration.
You will then register the user for those events.
You will then ask the user if they would like to register for any other events.
If the user says no, you will end the conversation. After ending the convesation show a summary of registration by calling function summary_registration.
If you encounter a situation for which you do not have an answer or a function call, then simply return a response back to the user that this enhancement is not quite available, and will be available in future.
Keep all the conversation related to the registration of the event only. Do not engage into personal or emotional conversations.
"""
welcome_prompt = """ 
welcome to the registration agent for the Seminar - Generative AI for Software Developers. 
How can I help you today ? Please provide your email to register for the event. 
You shall get a chance to add/delete/modify your registration. To finish the chat simply type - quit
"""



In [ ]:
# Define Tools to bind with LLM
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

@tool
def event_menu() -> str:
    """ Returns the list of events available for registration """
    return """
    Day 1:
    1. Introduction to Machine Learning Concepts , 9:00 AM - 10:00 AM
    2. Introduction to Generative AI and LLMs, 10:00 AM - 11:00 AM
    3. Agentic AI Fundamentals, 11:00 AM - 12:00 PM
    4. Lunch Break, 12:00 PM - 1:00 PM
    5. Building Agentic AI Applications Concepts, 1:00 PM - 2:00 PM
    6. Building Agentic AI Applications Hands-on, 2:00 PM - 3:00 PM
    7. Break, 3:00 PM - 3:30 PM
    8. Langgraph Concepts, 3:30 PM - 4:30 PM
    9. Langgraph Hands-on, 4:30 PM - 5:30 PM
    10. Wrap-up and Q&A, 5:30 PM - 6:00 PM
    Day 2:
    1. Introduction to Google ADK, 9:00 AM - 10:00 AM
    2. Google ADK Hands-on, 10:00 AM - 11:00 AM
    3. Popular Agentic Tools, 11:00 AM - 12:00 PM
    4. Lunch Break, 12:00 PM - 1:00 PM
    5. Responsible AI, 1:00 PM - 2:00 PM

    """

def decide_call_tool(state: RegisterState) -> Literal["tools", "human"]:
    """ Route between Tool calls or human node """
    # If no message found, throw value error
    if not state["messages"]:
        raise ValueError(f"No messages found in state : {state}")
    
    last_msg = state["messages"][-1]
    if len(last_msg.tool_calls) > 0:
        return "tools"
    else:
        return "human"

# Declare empty tool below, the implementation will stay under functions calls that can be added as node
# This is a recommended approach since tool call can not update the state, we need function for that


@tool
def add_registration(name: str, email: str, event: str) -> str:
    """ Adds the registration for the event 
    returns:
    Registration Added successfully in Not Confirmed status
    """

@tool
def confirm_registration() -> str:
    """ Confirms the registration for the event 
    returns:
    Registration is added with status : Confirmed for the registration that was added in the add_registration tool call
    """

@tool
def delete_registration(name: str, email: str, event: str) -> str:
    """ Deletes the registration for the event 
    returns:
    Registration is deleted successfully for the registration that was added in the add_registration tool call
    """
@tool
def get_registration(name: str, email: str) -> str:
    """ Gets the registration for the event 
    returns:
    Registration is fetched successfully for the registration that was added in the add_registration tool call
    """
@tool
def summary_registration(name: str, email: str) -> str:
    """ Summarizes the registration for the event 
    returns:
    Registration is fetched successfully for the registration that was added in the add_registration tool call
    """

# Define implementation of tool calls below as a node function
def register_event_node(state: RegisterState) -> RegisterState:
    """ This is the registration node where registration for events are added, updated, deleted and confirmed."""
    tool_msg = state["messages"][-1]
    registrations = state["registered_events"]
    outbound_response = []
    registration_done = False

    for tool_call in tool_msg.tool_calls:
        if tool_call["name"] == "add_registration":
            registrations.append(f"{tool_call['args']['event']} event is added successfully for {tool_call['args']['name']} with email {tool_call['args']['email']}. Pending Confirmation.")
            response_msg = "\n".join(registrations)
        elif tool_call["name"] == "confirm_registration":
            print("Confirming registration. You have: ")
            if not registrations:
                response_msg = "No registrations found to confirm."
                print(response_msg)
            for reg in registrations:
                print(reg)
            registration_done= True
            print("Does this look good to you ?")
        elif tool_call["name"] == "get_registration":
            response_msg = "\n".join(registrations) if registrations else "No registrations found."
        elif tool_call["name"] == "delete_registration":
            registrations.clear()
            response_msg = "All registrations deleted successfully."
        elif tool_call["name"] == "summary_registration":
            response_msg = "\n".join(registrations) if registrations else "No registrations found."
            registration_done= True
        else:
            response_msg = "Unknown tool call."
        
        outbound_response.append(
            ToolMessage(
                content=response_msg,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outbound_response, "registered_events": registrations, "finished_conversation": registration_done}


# Define method to route between chat and tool calls
def route_to_tools(state: RegisterState) -> RegisterState:
    """ Route to tools or chat based on the state """
    # If no message found, throw value error
    if not state["messages"]:
        raise ValueError(f"No messages found in state : {state}")
    
    last_msg = state["messages"][-1]
    if state["finished_conversation"]:
        return END
    elif len(last_msg.tool_calls) > 0:
        if any(tool["name"] in ["event_menu","add_registration", "confirm_registration", "delete_registration", "get_registration", "summary_registration"] for tool in last_msg.tool_calls):
            return "tools"
        else:
            return "register"
    else:
        # Call the human node to handle the conversation
        return "human"

In [ ]:
# Define Chatbot


# Bind tools to LLM
event_tools = [event_menu, add_registration, confirm_registration, delete_registration, get_registration, summary_registration]
tool_node = ToolNode(event_tools)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm_with_tools = llm.bind_tools(event_tools)

# Define Agent Node
def agent(state: RegisterState) -> RegisterState:
    """ This is a simplle chatbot that can help with registration """
    message_history = [SystemMessage(content=system_prompt)] + state["messages"]
    return {"messages": [llm.invoke(message_history)]}

# Define Human Node
def human_node(state: RegisterState) -> RegisterState:
    """ Display last model message to user and receive user input """
    last_message = state["messages"][-1]
    print(f"Assistant: {last_message.content}")
    user_msg = input("User: ")
    if user_msg.lower() == "quit":
        state["finished_conversation"] = True
        return state
    return state | {"messages": [HumanMessage(content=user_msg)]}


def welcome_node(state: RegisterState) -> RegisterState:
    """ Display welcome message to user and receive user input """
    if state["messages"]:
        new_message = [SystemMessage(content=system_prompt)] + state["messages"]
    else:
        new_message = [SystemMessage(content=welcome_prompt)]
    return state | {"messages": new_message}

def user_quit_node(state: RegisterState) -> Literal["agent", "__end__"]:
    """ Display goodbye message to user and receive user input """
    if state["finished_conversation"]:
        return END
    else :
        return "agent"
    return state | {"messages": [SystemMessage(content="Goodbye!")]}

# Define Agent Node With Tools
def agent_with_tools(state: RegisterState) -> RegisterState:
    """ This is agent with tools that can help with registration """
    defaults = {"registered_events":[], "finished_conversation": False}
    if state["messages"]:
        new_message = llm_with_tools.invoke([SystemMessage(content=system_prompt)] + state["messages"])
    else:
        new_message = AIMessage(content=welcome_prompt)
    return defaults | state | {"messages": [new_message]}
    


In [28]:
# Build StateGraph
register_graph_builder = StateGraph(RegisterState)
register_graph_builder.add_node("agent", agent_with_tools)
register_graph_builder.add_node("human", human_node)
register_graph_builder.add_node("tools", tool_node)
register_graph_builder.add_node("register", register_event_node)
register_graph_builder.add_conditional_edges("human", user_quit_node)
register_graph_builder.add_conditional_edges("agent", route_to_tools)
register_graph_builder.add_edge("tools", "agent")
register_graph_builder.add_edge("register", "agent")
register_graph_builder.add_edge(START, "agent")


register_graph = register_graph_builder.compile()

In [ ]:
# Invoke Chat 

config = {"recursion_limit": 100}
user_msg = input("User: ")
messages = [HumanMessage(content=user_msg)]

ai_events = register_graph.invoke({"messages": messages},config=config)

for msg in ai_events["messages"]:
     msg.pretty_print()